In [2]:
"""
지금부터 프로젝트 진행,
(가상) 온라인 쇼핑몰, 
실제로 우리가 이 쇼핑몰 비즈니스 팀이라는 가정하에  '여러가지 분석' 을 해보도록 하겠습니다.

중점을 두는 분석은
1. 현상태 파악
    - 고객 / 상품 분석
       : 단순히 막연히 '어제보다 고객이 줄었어',  '지난주보다 매출이 좀 늘었어'... 이런게 아니라.
       : 지난주 실제 매출이 얼마였고, (정확한 값!) 오늘의 고객증가량이 얼마였는지 (정확한 값 이 필요)
2. 의사결정에 도움되는 데이터 추출
   : 과거에는 '상부' 의 지시로 내려오거나 주로, '감 or 경험' 으로 많이 이루어졌죠.
   : 이제는 데이터가 있기에 '팩트' 에 맞는 결정을 내리는 거다 
"""
None

### 학습목표
1. 가상 쇼핑몰 고객 주문 데이터 파악하기
 - 현재 상황(데이터로 부터) 파악
 - 모델 수립 혹은 목표 설정

In [6]:
"""
↓ 아래 데이터는 이미 존재하는 공공 데이터임.
 영국의 모 온라인 리테일 사이트 의 1년간 주문 데이터
 건수는 약 50민건.
 출처 UCI  <-- ML 데이터 풍성
 원본의 형태는 엑셀파일인데,  CSV 로 변경하여 제공해드립니다.
"""
None

#### 데이터 셋 
 - 온라인 리테일 사이트의 2010/12 - 2011/12간의 주문 기록 데이터
 - 약 500,000건의 데이터 
 - 데이터 출처: [UCI ML Repository](https://archive.ics.uci.edu/ml/datasets/Online+Retail#)

In [7]:
import numpy as np
import pandas as pd

In [9]:
retail = pd.read_csv('../Data/OnlineRetail.csv')

In [15]:
# 컬럼 확인하기 위해서 .columns 속성 사용
retail.columns

# 8가지 정도의 속성.  쓱 보면 알수 있다.
# 딱히 어려운 내용은 없다

Index(['InvoiceNo', 'StockCode', 'Description', 'Quantity', 'InvoiceDate',
       'UnitPrice', 'CustomerID', 'Country'],
      dtype='object')

#### 컬럼 확인하기
 - columns 속성으로 확인
 - 컬럼
  - invoiceNo: 주문 번호
  - StockCode: 아이템 아이디
  - Description: 상품 설명
  - Quantity: 상품 주문 수량
  - InvoiceDate: 주문 시각
  - UnitPrice: 상품 '한개' 가격(동일한 통화)
  - CustomerID: 고객 아이디
  - Country: 고객 거주 지역(국가)

#### 데이터 살펴보기
 1. 데이터 분석의 가장 첫 단계
 2. 데이터를 대략적으로 파악 가능(타입, 저장된 형태)
 3. 데이터 cleansing 전략 수립

In [16]:
# head, info, describe()  <-- 3가지 함수로 간단히 알아보자

In [17]:
retail.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850.0,United Kingdom


In [18]:
# 관찰
# CustomerID 가 음.. 실수.   딱히 문제가 되는건 아니지만. 나중에 이걸 정수형으로 바꾸자.
# (굳이 실수로 표현할 필요 없다.)
# 그거 외에는 딱히 문제가 없어 보인다

In [20]:
retail.info()
# 어떤 값들이 '비어있는지' 확인해보자.

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   InvoiceNo    541909 non-null  object 
 1   StockCode    541909 non-null  object 
 2   Description  540455 non-null  object 
 3   Quantity     541909 non-null  int64  
 4   InvoiceDate  541909 non-null  object 
 5   UnitPrice    541909 non-null  float64
 6   CustomerID   406829 non-null  float64
 7   Country      541909 non-null  object 
dtypes: float64(2), int64(1), object(5)
memory usage: 33.1+ MB


In [ ]:
# ↑관찰
# 약 54만 데이터, 대부분의 필드가 같이 차 있는데..
# Description 과 CustomerID 가 비어 있는 것을 확인할수 있다!  주목!
# 현업에서는 ★ CustomerID 는 비어 있을수 없습니다.
# 분명히 Customer 테이블을 참조하는 FK 일텐데.  비어 있다는 것은 말이 안됩니다.
# 이런 상황은 사실 불가능  (그러나 고객 탈퇴 등을 Customer 에서 지운다면 이 또한 가능하다)

# 일단 우리가 다루는 이 상황에선 null 인 데이터는 의미가 없다 (누군지 모른다)
# 그래서 CustomerID 는 과감하게 제거하겠습니다.

In [22]:
# 기술통계량 확인, 숫자형 데이터에 대한 통계량 확인
retail.describe()

,Quantity,UnitPrice,CustomerID
count,541909.000000,541909.000000,406829.000000
mean,9.552250,4.611114,15287.690570
std,218.081158,96.759853,1713.600303
min,-80995.000000,-11062.060000,12346.000000
25%,1.000000,1.250000,13953.000000
50%,3.000000,2.080000,15152.000000
75%,10.000000,4.130000,16791.000000
max,80995.000000,38970.000000,18287.000000


In [26]:
# 관찰
# Quantity 
#      : 평균이 약 9.55 인데.   표준편차(std) 이 꽤 크게 나온다.. 음
#      : 헉 min 값이 -80995.000000 이다.  숫자니까. 어떠한 값도 가질수는 있겠지만.
#         과연 비즈니스 로직에 맞는 값이냐는 겁니다.
#         '수량' '가격' 은 항상 양수 여야 겠죠.
#         그런데 지금 보시면 Quantity 와 UnitPrice 에 둘다 음수가 있다.  말이 안된다.
#         이런 데이터도 과감하게 지워야 합니다. 
#         (현업에서는 거의 있을수 없는 불가능한 상황인거 같은데)

        

In [28]:
# 이제 이와 같이 살펴본 데이터를 근거로
# Cleaning 을 해야 할 2가지 포인트가 있죠↓

#### Data cleansing
 - null 데이터 처리
  - CustomerID 
 - Business 로직에 맞지 않은 데이터 처리
  - 음수의 아이템 수량
  - 가격이 0원 

In [29]:
retail.isnull()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...
541904,False,False,False,False,False,False,False,False
541905,False,False,False,False,False,False,False,False
541906,False,False,False,False,False,False,False,False
541907,False,False,False,False,False,False,False,False


In [31]:
# null 개수 총합 구하기
retail.isnull().sum()

InvoiceNo           0
StockCode           0
Description      1454
Quantity            0
InvoiceDate         0
UnitPrice           0
CustomerID     135080
Country             0
dtype: int64

In [ ]:
# 관찰
# Description      1454
# CustomerID     135080

# 우리는 거의 CustomerID 만 신경쓸거기 때문에 이것만 지우도록 하겠습니다.


#### null customerID 제거

In [32]:
retail['CustomerID']

0         17850.0
1         17850.0
2         17850.0
3         17850.0
4         17850.0
           ...   
541904    12680.0
541905    12680.0
541906    12680.0
541907    12680.0
541908    12680.0
Name: CustomerID, Length: 541909, dtype: float64

In [35]:
# null 이 아닌 것만 
pd.notnull(retail['CustomerID'])  # <-- True False 로 이루어졌다.  필터링 가능

0         True
1         True
2         True
3         True
4         True
          ... 
541904    True
541905    True
541906    True
541907    True
541908    True
Name: CustomerID, Length: 541909, dtype: bool

In [37]:
# 위 결과를 다시 넣으면 '필터링'
retail[pd.notnull(retail['CustomerID'])]

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
...,...,...,...,...,...,...,...,...
541904,581587,22613,PACK OF 20 SPACEBOY NAPKINS,12,12/9/2011 12:50,0.85,12680.0,France
541905,581587,22899,CHILDREN'S APRON DOLLY GIRL,6,12/9/2011 12:50,2.10,12680.0,France
541906,581587,23254,CHILDRENS CUTLERY DOLLY GIRL,4,12/9/2011 12:50,4.15,12680.0,France
541907,581587,23255,CHILDRENS CUTLERY CIRCUS PARADE,4,12/9/2011 12:50,4.15,12680.0,France


In [38]:
retail = retail[pd.notnull(retail['CustomerID'])]
len(retail)  # 몇개인지 확인

# 원래 54만개 --> 40만개로 줄음.

406829

#### 비지니스 로직에 맞지 않은 데이터 제거
 - 수량, 가격 > 0

In [39]:
retail['Quantity']

0          6
1          6
2          8
3          6
4          6
          ..
541904    12
541905     6
541906     4
541907     4
541908     3
Name: Quantity, Length: 406829, dtype: int64

In [40]:
retail['Quantity'] > 0

0         True
1         True
2         True
3         True
4         True
          ... 
541904    True
541905    True
541906    True
541907    True
541908    True
Name: Quantity, Length: 406829, dtype: bool

In [41]:
retail[retail['Quantity'] > 0]

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
...,...,...,...,...,...,...,...,...
541904,581587,22613,PACK OF 20 SPACEBOY NAPKINS,12,12/9/2011 12:50,0.85,12680.0,France
541905,581587,22899,CHILDREN'S APRON DOLLY GIRL,6,12/9/2011 12:50,2.10,12680.0,France
541906,581587,23254,CHILDRENS CUTLERY DOLLY GIRL,4,12/9/2011 12:50,4.15,12680.0,France
541907,581587,23255,CHILDRENS CUTLERY CIRCUS PARADE,4,12/9/2011 12:50,4.15,12680.0,France


In [42]:
retail = retail[retail['Quantity'] > 0]
retail = retail[retail['UnitPrice'] > 0]

len(retail)  # 결과 확인

397884

In [43]:
# cleasing 을 했으니
# 다시 데이터를 확인해보자

In [45]:
retail.info()
# 이제 null 이 없음을 확인할수 있다.

<class 'pandas.core.frame.DataFrame'>
Int64Index: 397884 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   InvoiceNo    397884 non-null  object 
 1   StockCode    397884 non-null  object 
 2   Description  397884 non-null  object 
 3   Quantity     397884 non-null  int64  
 4   InvoiceDate  397884 non-null  object 
 5   UnitPrice    397884 non-null  float64
 6   CustomerID   397884 non-null  float64
 7   Country      397884 non-null  object 
dtypes: float64(2), int64(1), object(5)
memory usage: 27.3+ MB


In [47]:
retail.describe()
# 수량, 값  둘다 '음수' 가 없어졌슴을 확인 가능하다

,Quantity,UnitPrice,CustomerID
count,397884.000000,397884.000000,397884.000000
mean,12.988238,3.116488,15294.423453
std,179.331775,22.097877,1713.141560
min,1.000000,0.001000,12346.000000
25%,2.000000,1.250000,13969.000000
50%,6.000000,1.950000,15159.000000
75%,12.000000,3.750000,16795.000000
max,80995.000000,8142.750000,18287.000000


In [ ]:
# 이제
# 데이터 분석에서 중요한 것중에 하나가 
# '메모리' 를 얼마나 효율적으로 쓸수 있느냐 입니다.
# 

#### 데이터 타입 변경
 - 메모리 효율화
 - 올바른 데이터 타입 매칭

In [49]:
retail['CustomerID']

0         17850.0
1         17850.0
2         17850.0
3         17850.0
4         17850.0
           ...   
541904    12680.0
541905    12680.0
541906    12680.0
541907    12680.0
541908    12680.0
Name: CustomerID, Length: 397884, dtype: float64

In [48]:
retail['CustomerID'].astype(np.int32)  # Customer 가 39만명 정도 이기 때문에 int32 이면 적당.

0         17850
1         17850
2         17850
3         17850
4         17850
          ...  
541904    12680
541905    12680
541906    12680
541907    12680
541908    12680
Name: CustomerID, Length: 397884, dtype: int32

In [50]:
retail['CustomerID'] = retail['CustomerID'].astype(np.int32)

In [53]:
retail.info()
# CustomerID 의 dtype 이 int32로 바뀌었다.
# 메모리 용량도 대략 27.3M 에서 25.8M 로 바뀐 것을 확인해보자

# 상황에 따라 타입변경을 통해 메모리를 효율적으로 사용할수 있다.

<class 'pandas.core.frame.DataFrame'>
Int64Index: 397884 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   InvoiceNo    397884 non-null  object 
 1   StockCode    397884 non-null  object 
 2   Description  397884 non-null  object 
 3   Quantity     397884 non-null  int64  
 4   InvoiceDate  397884 non-null  object 
 5   UnitPrice    397884 non-null  float64
 6   CustomerID   397884 non-null  int32  
 7   Country      397884 non-null  object 
dtypes: float64(1), int32(1), int64(1), object(5)
memory usage: 25.8+ MB


In [55]:
# 이제 cleasing 은 거의 마무리가 되었고

# 마지막으로 새로운 컬럼을 추가 합니다.

# '전체 사용자' 가 지불한 금액은 없다.

#### 새로운 컬럼 추가
 - Quantity * UnitPrice는 고객의 총 지출 비용(CheckoutPrice)

In [57]:
retail['CheckoutPrice'] = retail['UnitPrice'] * retail['Quantity']
retail.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,CheckoutPrice
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850,United Kingdom,15.30
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850,United Kingdom,20.34
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850,United Kingdom,22.00
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850,United Kingdom,20.34
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850,United Kingdom,20.34


## 정제된 데이터 저장

In [59]:
retail.to_csv('../Data/OnlineRetailClean.csv')
# 앞으로 이 데이터를 사용하겠습니다

---
## 미니 프로젝트를 통한 데이터 분석의 목표
 1. 매출 분석
 2. 고객 분석
  - 우수고객 선별
  - 고객 리텐션 분석
 3. push notification 실행 의사 결정 하기

In [ ]:
# 우리가 할 데이터분석 목표
# 1. 현상태 파악
#      매출분석
#      고객분석
        
# 2. 현재 존재 데이터로 의사 결정하기